<a href="https://colab.research.google.com/github/pedrobribeiro2/99Hunters_Projects/blob/main/2023%2012%20Retrospectiva_Hunters_Processamento%20de%20Reports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# Adjust the file path below to the location of your file in Google Drive
file_path = '/content/drive/MyDrive/Pedro Marketing/2023 12 99Hunters Wrapped/2023_12_15_hunter_jobs_report.csv'  # Replace with your file path in Google Drive

# Load the dataset
data = pd.read_csv(file_path)

# Display the first few rows to verify the data
print(data.head())


   ID_Hunter_Job  ID_Hunter  ID_Job       Job Executive      Job Manager  \
0          26405       2719    4477  Danielle Calheiros  Victor Valentim   
1          26406       4294    4477  Danielle Calheiros  Victor Valentim   
2          26407       4442    4477  Danielle Calheiros  Victor Valentim   
3          26408       3286    4477  Danielle Calheiros  Victor Valentim   
4          26409       6032    4477  Danielle Calheiros  Victor Valentim   

                        Hunter_Name Hunter is Team99 Company_Name  \
0                      Pamela Simas           Hunter          GPA   
1                     Jessica Silva           Hunter          GPA   
2                    Luana Carvalho           Hunter          GPA   
3  Chiara Carolina Martins de Souza           Hunter          GPA   
4                         Giselle .           Hunter          GPA   

                                     Job_Name  Job Code  ... recommended  \
0  Pessoa Coordenadora de Planejamento Sênior    216

In [ ]:
# Group the data by 'ID_Hunter' and calculate the necessary statistics
grouped_data = data.groupby('ID_Hunter').agg(
    Full_Name=('Hunter_Name', 'first'),  # Get the first name in each group
    Total_Recommendations=('recommended', 'sum'),
    Total_Shortlisted=('shortlisted', 'sum'),
    Total_Interviewed=('interviewed', 'sum'),
    Total_Hired=('hired', 'sum'),
    Earnings_2023=('Fee', 'sum')  # Assuming 'Fee' column represents earnings
).reset_index()

# Display the processed data
print(grouped_data.head())


   ID_Hunter               Full_Name  Total_Recommendations  \
0          4        Luciano Montezzo                   28.0   
1         46           Maib Oliveira                    3.0   
2         63           Felipe Arruda                    0.0   
3         65          Cristina Cappi                    0.0   
4         66  Marta Helena Gonçalves                    0.0   

   Total_Shortlisted  Total_Interviewed  Total_Hired  Earnings_2023  
0                5.0                2.0          0.0            200  
1                1.0                1.0          0.0            100  
2                0.0                0.0          0.0              0  
3                0.0                0.0          0.0              0  
4                0.0                0.0          0.0              0  


In [ ]:
# Filtrando os dados para incluir apenas os status desejados
status_validos = ['open', 'interviews', 'shortlist', 'placement', 'adv_interviews', 'stand_by']
data_filtrada = data[data['HunterJob Status'].isin(status_validos)]

# Contabilizando o número de jobs trabalhados por cada Hunter
jobs_contabilizados = data_filtrada.groupby('ID_Hunter')['ID_Job'].nunique().reset_index()
jobs_contabilizados.rename(columns={'ID_Job': 'Total_Jobs_Trabalhados'}, inplace=True)

# Mesclando os dados contabilizados com os dados agrupados anteriores
final_df = pd.merge(grouped_data, jobs_contabilizados, on='ID_Hunter', how='left')

# Mesclando os dados contabilizados com os dados agrupados anteriores
grouped_data = pd.merge(grouped_data, jobs_contabilizados, on='ID_Hunter', how='left')

# Preenchendo valores ausentes com 0, caso algum Hunter não tenha jobs contabilizados
grouped_data['Total_Jobs_Trabalhados'].fillna(0, inplace=True)


In [ ]:
# Importar a biblioteca necessária
import calendar

# Converter a coluna 'Created_at' para datetime
data['Created_at'] = pd.to_datetime(data['Created_at'], format='%m/%d/%Y-%H:%M')

# Extrair o mês e o ano de cada job
data['Month'] = data['Created_at'].dt.month
data['Year'] = data['Created_at'].dt.year

# Filtrar os dados para incluir apenas os jobs dos meses e status desejados
data_filtered = data[(data['HunterJob Status'].isin(status_validos)) & (data['Year'] == 2023)]

# Contabilizar os jobs por mês para cada hunter
monthly_activity = data_filtered.groupby(['ID_Hunter', 'Month']).size().reset_index(name='Jobs_Per_Month')

# Mapeamento de número do mês para nome do mês em português
meses_pt_br = {
    1: 'Janeiro', 2: 'Fevereiro', 3: 'Março', 4: 'Abril', 5: 'Maio', 6: 'Junho',
    7: 'Julho', 8: 'Agosto', 9: 'Setembro', 10: 'Outubro', 11: 'Novembro', 12: 'Dezembro'
}

# Encontrar os três meses mais ativos para cada hunter
def top_3_months(df):
    top_months = df.sort_values('Jobs_Per_Month', ascending=False).head(3)
    # Ordenar os meses cronologicamente antes de juntá-los em uma string
    top_months_sorted = top_months.sort_values('Month')
    return ', '.join(top_months_sorted['Month'].apply(lambda x: meses_pt_br[x]))

# Aplicação da função top_3_months e criação de top_months_df
top_months_df = monthly_activity.groupby('ID_Hunter').apply(top_3_months).reset_index(name='Top_3_Active_Months')

# Mesclagem com grouped_data
grouped_data = pd.merge(grouped_data, top_months_df, on='ID_Hunter', how='left')

# Preenchimento de valores ausentes
grouped_data['Top_3_Active_Months'].fillna('No active months', inplace=True)

In [ ]:
# Rank hunters based on Total Recommendations in descending order
grouped_data['Ranking'] = grouped_data['Total_Recommendations'].rank(ascending=False, method='min')


In [ ]:
import numpy as np

# Calculate the percentile rank
grouped_data['Percentile'] = grouped_data['Total_Recommendations'].rank(pct=True, ascending=False)

# Convert the percentile to a more readable format (e.g., top 10%)
grouped_data['Percentile'] = np.ceil(grouped_data['Percentile'] * 100).astype(int)


In [ ]:
# Converter colunas para tipo inteiro
colunas_numericas = ['Total_Jobs_Trabalhados', 'Total_Recommendations', 'Total_Shortlisted', 'Total_Interviewed', 'Total_Hired', 'Ranking']
grouped_data[colunas_numericas] = grouped_data[colunas_numericas].fillna(0).astype(int)

# Formatar 'Earnings_2023' como moeda BRL
grouped_data['Earnings_2023'] = grouped_data['Earnings_2023'].apply(lambda x: f'R$ {x:,.2f}'.replace(',', 'X').replace('.', ',').replace('X', '.'))

# Verificar os tipos de dados após conversão
print(grouped_data.dtypes)


ID_Hunter                  int64
Full_Name                 object
Total_Recommendations      int64
Total_Shortlisted          int64
Total_Interviewed          int64
Total_Hired                int64
Earnings_2023             object
Total_Jobs_Trabalhados     int64
Top_3_Active_Months       object
Ranking                    int64
Percentile                 int64
dtype: object


In [ ]:
# Create the final DataFrame with the new column
final_df = grouped_data[['ID_Hunter', 'Full_Name', 'Total_Jobs_Trabalhados','Total_Recommendations', 'Total_Shortlisted', 'Total_Interviewed', 'Total_Hired', 'Earnings_2023', 'Ranking', 'Percentile', 'Top_3_Active_Months']]

# Sort the DataFrame by 'Ranking' in ascending order (top rank first)
final_df = final_df.sort_values(by='Ranking', ascending=True)

# Export to CSV
final_df.to_csv('Retrospectiva_Hunters_2023_12_15_VF.csv', index=False)


In [ ]:
from google.colab import files
files.download('Retrospectiva_Hunters_2023_12_15_VF.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>